In [6]:
import numpy as np
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def generate_data_first_example(n=750, d=10, sigma=0.5):
    """
    Generate data for the first example:
    m(x) = 5 * x1^2 * x2^2 with d=10, xi ~ Uniform(0, 1), and noise ~ N(0, sigma^2).
    """
    # Generate random samples from Uniform(0, 1)
    X = np.random.uniform(0, 1, size=(n, d))
    
    # True function: m(x) = 5 * x1^2 * x2^2
    m = 5 * X[:, 0]**2 * X[:, 1]**2
    
    # Add Gaussian noise
    Y = m + np.random.normal(0, sigma, size=n)
    
    return X.T, Y

def generate_data_second_example(n=750, d=20, sigma=1.0):
    """
    Generate data for the second example:
    m(x) = 2 * (x1 + 1)^3 + 2 * sin(10 * x2) with d=20, xi ~ Uniform(0, 1), and noise ~ N(0, sigma^2).
    """
    # Generate random samples from Uniform(0, 1)
    X = np.random.uniform(0, 1, size=(n, d))
    
    # True function: m(x) = 2 * (x1 + 1)^3 + 2 * sin(10 * x2)
    m = 2 * (X[:, 0] + 1)**3 + 2 * np.sin(10 * X[:, 1])
    
    # Add Gaussian noise
    Y = m + np.random.normal(0, sigma, size=n)
    
    return X.T, Y

sigma1 = 0.5
X1_train, Y1_train = generate_data_first_example(sigma=sigma1)
print("First Example:")
print("X1_train shape:", X1_train.shape)
print("Y1_train shape:", Y1_train.shape)

# Second example
sigma2 = 1.0
X2_new, Y2_new = generate_data_second_example(sigma=sigma2)
print("\nSecond Example:")
print("X2_new shape:", X2_new.shape)
print("Y2_new shape:", Y2_new.shape)

test_X1 = np.array([0.5,] * X1_train.shape[0])
test_X2 = np.array([0.5,] * X2_new.shape[0])


First Example:
X1_train shape: (10, 750)
Y1_train shape: (750,)

Second Example:
X2_new shape: (20, 750)
Y2_new shape: (750,)


In [ ]:
train_size = int(10e3) # int(10e4) * 5
test_size = 100
X1_train, Y1_train = generate_data_first_example(n=train_size, sigma=sigma1)
X1_test, Y1_test = generate_data_first_example(n=test_size,sigma=sigma1)

# Train size is 10^3, test size is 100
# Time taken for Rodeo:  1.8304240703582764
# 0.4761482444116567 0.36334789023536473 0.656648952366093

# Train size is 10^4, test size is 100
# Time taken for Rodeo:  17.49112296104431
# 0.42113303924887197 0.2681481806486581 0.7190478643011831

# Train size is 5 * 10^4, test size is 100
# Time taken for Rodeo:  129.53368997573853
# 0.3378923726690212 0.1796326447472333 0.8485084769651323



In [56]:
import importlib
import kernels
import rodeo_regression
from rodeo_regression import rodeo_regression
from model import model
import time

# Create a list of kernels for each dimension
kernels_list = [kernels.gaussian_kernel() for i in range(X1_train.shape[0])] 

# Initialize the model with the kernels
stat_model = model(kernels_list)
from rodeo_regression import rodeo_regression
rodeo = rodeo_regression(stat_model=stat_model, data=X1_train, responses=Y1_train, given_sigma2= sigma1 ** 2, verbose=False, regression_type="constant")
# Define grid points for evaluation

start_time = time.time()
rodeo_predictions = []
rodeo_bw = []
for idx in range(X1_test.shape[1]):
    h_star, m_hat = rodeo.local_rodeo(point=X1_test[:, idx], beta=0.8)
    rodeo_predictions.append(m_hat)
    rodeo_bw.append(h_star)
Rodeo_time = time.time() - start_time
print("Time taken for Rodeo: ", Rodeo_time)

print(mean_absolute_error(Y1_test,rodeo_predictions), mean_squared_error(Y1_test,rodeo_predictions), r2_score(Y1_test,rodeo_predictions))

Time taken for Rodeo:  11.663727045059204
0.3959616160944484 0.24016171948515538 0.7701534551178165


In [57]:
rodeo_bw

[array([0.05174006, 0.08065391, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.05174006, 0.15752717, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.06467507, 0.15752717, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.06467507, 0.15752717, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.19737266, 0.06452313, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.05174006, 0.15752717, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.19737266, 0.06452313, 0.75133607, 0.75128736, 0.75119333,
        0.75187478, 0.75184392, 0.75052292, 0.75298582, 0.75076751]),
 array([0.1263185 , 0.06452313, 0.

In [52]:

import time
model_name = []
mae = []
mse = []
r2 = []
model_training_time = []

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'SGD Regressor': SGDRegressor(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    # 'Support Vector Regressor': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Dummy Regressor': DummyRegressor(strategy='mean')
}

param_grids = {
    'Linear Regression': {},
    'SGD Regressor': {'alpha': [1e-3], 'max_iter': [1000]},
    'Ridge': {'alpha': [1.0]},
    'Lasso': {'alpha': [0.1]},
    'ElasticNet': {'alpha': [0.1], 'l1_ratio': [0.5]},
    'Support Vector Regressor': {'C': [1], 'epsilon': [0.2]},
    'Decision Tree': {'max_depth': [10], 'min_samples_split': [5]},
    'Random Forest': {
        'n_estimators': [100],
        'max_depth': [20],
        'min_samples_split': [5],
        'max_features': ['sqrt']
    },
    'Dummy Regressor': {}
}

fitted_models = {}
# Loop over each model with hyperparameter tuning
for name, model in models.items():
    print(f"Training {name}...")
    start_time = time.time()
    
    # Set up GridSearchCV with the model and its parameter grid
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X1_train.T, Y1_train)
    
    # Record training time
    model_training_time.append(time.time() - start_time)
    
    # Best model after CV
    best_model = grid_search.best_estimator_
    
    # Make predictions on the test set
    Y_pred = best_model.predict(X1_test.T)
    fitted_models[name] = best_model
    # Calculate metrics on the test set
    model_name.append(name)
    mae.append(mean_absolute_error(Y1_test, Y_pred))
    mse.append(mean_squared_error(Y1_test, Y_pred))
    r2.append(r2_score(Y1_test, Y_pred))

model_name.append("Rodeo")
mae.append(mean_absolute_error(Y1_test, rodeo_predictions))
mse.append(mean_squared_error(Y1_test, rodeo_predictions))
r2.append(r2_score(Y1_test, rodeo_predictions))
model_training_time.append(Rodeo_time)

# Compile results into a DataFrame
results_df = pd.DataFrame({
    'Model': model_name,
    'MAE': mae,
    'MSE': mse,
    'R^2': r2,
    'Training Time (s)': model_training_time
})

results_df = results_df.sort_values(by='R^2', ascending=False)
results_df

Training Linear Regression...
Training SGD Regressor...
Training Ridge...
Training Lasso...
Training ElasticNet...
Training Decision Tree...
Training Random Forest...
Training Dummy Regressor...


,Model,MAE,MSE,R^2,Training Time (s)
8,Rodeo,0.337892,0.179633,0.848508,129.533690
5,Decision Tree,0.343085,0.182914,0.845741,11.420027
6,Random Forest,0.337028,0.183821,0.844977,397.818044
0,Linear Regression,0.542893,0.486055,0.590090,2.121523
2,Ridge,0.542894,0.486060,0.590086,0.242749
1,SGD Regressor,0.544353,0.488439,0.588080,1.605991
4,ElasticNet,0.647057,0.803617,0.322277,0.720603
3,Lasso,0.681648,0.908409,0.233901,0.656118
7,Dummy Regressor,0.778770,1.211961,-0.022096,0.688532


In [23]:
from statsmodels.nonparametric.kernel_regression import KernelReg
from sklearn.model_selection import GridSearchCV
import time
import numpy as np

# Define a wrapper class for KernelReg to fit into the scikit-learn framework
class StatsmodelsKernelReg:
    def __init__(self, bw='cv_ls', var_type='c', reg_type='ll'):
        self.bw = bw
        self.var_type = var_type
        self.reg_type = reg_type
        self.model = None

    def fit(self, X, y):
        self.model = KernelReg(endog=y, exog=X, var_type=self.var_type, reg_type=self.reg_type, bw=self.bw)
        return self

    def predict(self, X):
        preds, marginal_effect = self.model.fit(X)
        return preds, marginal_effect

    # Add get_params method
    def get_params(self, deep=True):
        return {'bw': self.bw, 'var_type': self.var_type, 'reg_type': self.reg_type}

    # Add set_params method
    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self

X_train = X1_train.T
Y_train = Y1_train

# Instantiate the Kernel Regression model with the specified parameters
kernel_reg_model = StatsmodelsKernelReg(bw='cv_ls', var_type='c' * X_train.shape[1], reg_type='lc')


start_time = time.time()
kernel_reg_model.fit(X_train, Y_train)
keg_training_time = time.time() - start_time

Y_pred,marginal_effect = kernel_reg_model.predict(X1_test.T)

# Calculate metrics on the test set
kernel_reg_mae = mean_absolute_error(Y1_test, Y_pred)
kernel_reg_mse = mean_squared_error(Y1_test, Y_pred)
kernel_reg_r2 = r2_score(Y1_test, Y_pred)
print("MAE, MSE, R^2\n", kernel_reg_mae,kernel_reg_mse,kernel_reg_r2)
row_to_add = ['Fixed Keg Model', kernel_reg_mae, kernel_reg_mse, kernel_reg_r2, keg_training_time]
results_df.loc[len(results_df)] = row_to_add
results_df = results_df.sort_values(by='R^2', ascending=False)
results_df

MAE, MSE, R^2
 0.4124978799133742 0.2738111819044289 0.7429404454741035


,Model,MAE,MSE,R^2,Training Time (s)
9,Fixed Keg Model,0.412498,0.273811,0.742940,218.807878
8,Rodeo,0.417308,0.293390,0.724559,0.100000
6,Random Forest,0.479566,0.379216,0.643985,0.346132
0,Linear Regression,0.584458,0.559328,0.474892,0.055466
2,Ridge,0.584274,0.560518,0.473774,0.027301
5,Decision Tree,0.609150,0.576579,0.458696,0.018106
1,SGD Regressor,0.606991,0.624051,0.414128,0.028599
4,ElasticNet,0.656117,0.761531,0.285059,0.014298
3,Lasso,0.682063,0.831569,0.219306,0.014699
7,Dummy Regressor,0.791951,1.102965,-0.035486,0.017475


In [26]:
kernel_reg_model.model.bw.round(3)

array([6.600000e-02, 6.500000e-02, 3.202137e+03, 1.593149e+03,
       2.926110e+02, 2.816567e+03, 2.076340e+02, 2.351482e+03,
       5.050000e-01, 6.778175e+03])